In [1]:
# Import libraries
import statsapi
import requests as re
from urllib.request import urlopen
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm
import time
import more_itertools as mit
from bs4 import BeautifulSoup
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
pd.set_option('display.max_columns', 500)

In [2]:
# Web-scrape headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}

# Get team schedules

In [2]:
# Get Schedule
def get_schedule(year):
    df = pd.DataFrame(statsapi.schedule(start_date='01/01/'+year,end_date='12/31/'+year))

    # Get regular season games
    r = df[df['game_type']=='R']

    # Get final games
    r = r[~r['winning_pitcher'].isna()]

    # Remove games delayed to another day
    index = r['game_id'].value_counts().reset_index()
    r=r[~r['game_id'].isin(index[index['game_id'] > 1]['index'])]
    r=r[r['doubleheader'] == 'N']
    return r

In [33]:
# Remove columns we dont need
games = r.drop(columns=['game_datetime','game_type','status','doubleheader','game_num','home_probable_pitcher', 'away_probable_pitcher',
       'home_pitcher_note', 'away_pitcher_note', 'away_score', 'home_score',
       'current_inning', 'inning_state', 'venue_id', 'venue_name', 'summary',
       'winning_team', 'losing_team', 'winning_pitcher', 'losing_pitcher',
       'save_pitcher'])

# Get game info

In [3]:
# Get game info for each game
info = {'game_id':[], 'HP':[], 'temp':[], 'weather':[], 'mag':[], 'dir':[], 'length':[], 'att':[], 'daytime':[], 'delay':[]}

for gameid in tqdm(games['game_id']):
    #try:
    
        info['game_id'].append(gameid)
        game = statsapi.boxscore_data(gameid)
        
        # Get game info wind, att, hp...
        infobox = game['gameBoxInfo']

        HP = [i['value']  for i in infobox if i['label'] == 'Umpires'][0].split('.')[0][4:]
        info['HP'].append(HP)

        temp, weather = [i['value']  for i in infobox if i['label'] == 'Weather'][0].split(',')
        info['temp'].append(temp.split(' ')[0])
        info['weather'].append(weather[1:-1])

        wind = [i['value']  for i in infobox if i['label'] == 'Wind']
        if len(wind) == 0:
            info['mag'].append(np.nan)
            info['dir'].append(np.nan)
        else:
            wind = wind[0]
            mag, dire = wind.split(', ')
            info['mag'].append(mag)
            info['dir'].append(dire)
        
        gametime = [i['value']  for i in infobox if i['label'] == 'T']
        if len(gametime) == 0:
            info['length'].append(np.nan)
        else:
            gametime = gametime[0]
            if 'delay' in gametime:
                info['delay'].append(1)
                hour, m = gametime.split(' ')[0].split(':')

            else:
                info['delay'].append(0)
                hour, m = gametime.split(':')

            length = int(hour)* 60 + int(m[:2])
            info['length'].append(length)

        att = [i['value']  for i in infobox if i['label'] == 'Att']
        if len(att) == 0:
            info['att'].append(np.nan)
        else:
            info['att'].append(att[0][:-1].replace(',',''))

        daytime = [i['value']  for i in infobox if i['label'] == 'First pitch'][0][:-1]
        info['daytime'].append(daytime)
 
        time.sleep(1)
    #except IndexError:
        #print(gameid)

100%|████████████████████████████████████████████████████████████████████████████| 2383/2383 [1:05:45<00:00,  1.66s/it]


In [4]:
# Convert dictionary to dataframe
info = pd.DataFrame(info)

In [5]:
# Merge to get team and pitcher info and save it to save memory
games.merge(info, left_on=['game_id'], right_on=['game_id']).to_csv('Data/2012.csv',index=False)

In [47]:
# Read file and process it further
df = pd.read_csv('Data/2013.csv')

,game_id,game_date,away_name,home_name,away_id,home_id,HP,temp,weather,mag,dir,length,att,daytime,delay,away_LOB,away_RISP,away_GIDP,away_2outrunners,away_2outrbi,home_LOB,home_RISP,home_GIDP,home_2outrunners,home_2outrbi,away_runs,away_doubles,away_triples,away_homeRuns,away_strikeOuts,away_baseOnBalls,away_hits,away_avg,away_atBats,away_obp,away_slg,away_ops,away_stolenBases,away_rbi,away_leftOnBase,home_runs,home_doubles,home_triples,home_homeRuns,home_strikeOuts,home_baseOnBalls,home_hits,home_avg,home_atBats,home_obp,home_slg,home_ops,home_stolenBases,home_rbi,home_leftOnBase
0,346787,2013-03-31,Texas Rangers,Houston Astros,140,117,Sam Holbrook,73,Roof Closed,0 mph,None.,180,41307,7:08 PM,0,6,2-for-6,0,3,2,3,4-for-6,0,1,6,2,0,0,0,7,3,6,0.188,32,0.257,0.188,0.445,0,2,12,8,0,2,1,13,3,9,0.281,32,0.343,0.500,0.843,0,7,7
1,346187,2013-04-01,Miami Marlins,Washington Nationals,146,120,Jeff Kellogg,61,Partly Cloudy,15 mph,Out To RF.,130,45274,1:09 PM,0,3,1-for-4,0,1,0,4,0-for-4,1,2,1,0,1,0,0,6,1,3,0.103,29,0.133,0.138,0.271,0,0,8,2,0,0,2,5,2,5,0.192,26,0.250,0.423,0.673,0,2,8
2,346755,2013-04-01,Boston Red Sox,New York Yankees,111,147,Ted Barrett,62,Partly Cloudy,17 mph,R To L.,217,49514,1:10 PM,0,13,6-for-18,0,8,6,8,2-for-9,1,3,2,8,1,1,0,8,8,13,0.325,40,0.438,0.400,0.838,0,8,26,2,1,0,0,10,4,6,0.182,33,0.270,0.212,0.482,0,2,21
3,346189,2013-04-01,San Diego Padres,New York Mets,135,121,Tim McClelland,61,Cloudy,18 mph,Out To CF.,181,41053,1:13 PM,0,6,1-for-7,0,3,1,8,7-for-14,0,4,9,2,1,0,1,6,2,4,0.129,31,0.206,0.258,0.464,0,2,10,11,2,0,1,10,5,13,0.342,38,0.419,0.474,0.892,2,11,14
4,346184,2013-04-01,Chicago Cubs,Pittsburgh Pirates,112,134,Tom Hallion,41,Cloudy,12 mph,Out To CF.,179,39078,1:41 PM,0,5,0-for-8,0,4,0,6,1-for-7,0,3,0,3,2,0,1,15,1,6,0.182,33,0.229,0.333,0.562,2,3,12,1,1,0,0,10,2,3,0.097,31,0.176,0.129,0.306,1,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,349168,2013-09-29,San Diego Padres,San Francisco Giants,135,137,Quinn Wolcott,67,Cloudy,9 mph,Out To RF.,197,41495,1:06 PM,0,4,2-for-6,1,3,0,10,5-for-15,1,4,3,6,2,1,2,8,4,7,0.245,33,0.308,0.378,0.686,0,6,9,7,3,0,1,8,5,11,0.260,34,0.320,0.381,0.702,1,7,18
2365,349162,2013-09-29,Colorado Rockies,Los Angeles Dodgers,115,119,Bob Davidson,85,Sunny,5 mph,Out To CF.,188,52396,1:10 PM,0,12,4-for-14,2,6,0,7,1-for-9,0,5,0,2,1,0,0,10,5,11,0.270,35,0.323,0.418,0.741,1,2,30,1,1,0,0,13,4,5,0.264,31,0.326,0.396,0.722,0,1,18
2366,349169,2013-09-29,Washington Nationals,Arizona Diamondbacks,120,109,Tripp Gibson,74,Roof Closed,0 mph,None.,149,30390,1:10 PM,0,7,2-for-5,1,2,0,7,3-for-8,0,3,1,2,0,1,0,5,1,8,0.251,35,0.313,0.398,0.710,1,2,15,3,0,0,0,4,1,7,0.259,31,0.323,0.391,0.715,0,3,15
2367,349173,2013-09-29,Oakland Athletics,Seattle Mariners,133,136,Marvin Hudson,56,Roof Closed,0 mph,None.,188,17081,1:10 PM,0,7,6-for-14,0,5,2,9,0-for-5,0,4,0,9,5,0,0,9,6,10,0.254,37,0.327,0.419,0.745,0,9,17,0,0,0,0,10,4,5,0.237,32,0.306,0.390,0.695,0,0,19


# Get team abbreviations

In [81]:
# Get team abbreviations
d = {'teamid':[],'teamabb':[]}
for team in df['away_id'].unique():
    d['teamabb'].append(statsapi.lookup_team(team)[0]['fileCode'].upper())
    d['teamid'].append(team)

In [84]:
# Save file
pd.DataFrame(d).to_csv('Data/teamabbre.csv',index=False)

In [58]:
# convert date to days after 01/01/2013 so we can group them in series
df['days'] = df['game_date'].apply(lambda x: (datetime.datetime.strptime(x,'%Y-%m-%d') - datetime.datetime.strptime('2013-01-01','%Y-%m-%d')).days)

In [59]:
# prepare the columns of a dataframe containing a series of 3 games as a data row
cols1 = df.columns.tolist()
cols2 = df.columns.tolist()
for i in range (len(cols1)):
    cols1[i] += '_1'
for i in range (len(cols2)):
    cols2[i] += '_2'
cols = cols1 + cols2 + df.columns.tolist()

In [60]:
# Format the games into series of 3 games
train = pd.DataFrame()
for away in df['away_id'].unique():
    temp = df[df['away_id'] == away]
    for home in df['home_id'].unique():
        temp2 = temp[temp['home_id']==home].sort_values('game_date')
        
        for series in [list(group) for group in mit.consecutive_groups(temp2['days'].tolist())]:
            if len(series) == 3:
                thisseries = temp2[temp2['days'].isin(series)].sort_values('game_date')
                s = pd.concat([thisseries.iloc[0],thisseries.iloc[1],thisseries.iloc[2]]).to_frame().transpose()
                s.columns = cols
                train = pd.concat([train, s])

In [20]:
# save them as initial training data
train.to_csv('train/2014.csv',index=False)

# Get pitching and batting info

In [33]:
# collection of all games from 2012 to 2019
df= pd.concat([pd.read_csv('Data/2012.csv'),pd.read_csv('Data/2013.csv'),
               pd.read_csv('Data/2014.csv'),pd.read_csv('Data/2015.csv'),
               pd.read_csv('Data/2016.csv'),pd.read_csv('Data/2017.csv'),
               pd.read_csv('Data/2018.csv'),pd.read_csv('Data/2019.csv')])
df = df[['game_id','game_date','away_name','home_name','away_id','home_id','temp','daytime']]

In [7]:
# team abbreviations file
d = pd.read_csv('Data/teamabbre.csv')

In [8]:
# Merge to get team abbreviations
df = df.merge(d,how='left',left_on=['away_id'],right_on=['teamid']).merge(d,how='left',left_on=['home_id'],right_on=['teamid'],suffixes=['_away','_home'])

In [9]:
# Drop duplicated team ids 
df.drop(columns=['away_id','home_id'],inplace=True)

In [10]:
# Observe the missing values for each column
df.isna().sum()

game_id         0
game_date       0
away_name       0
home_name       0
temp            0
daytime         0
teamid_away     0
teamabb_away    0
teamid_home     0
teamabb_home    0
dtype: int64

In [11]:
df

,game_id,game_date,away_name,home_name,temp,daytime,teamid_away,teamabb_away,teamid_home,teamabb_home
0,317775,2012-03-28,Seattle Mariners,Oakland Athletics,72,7:09 PM,136,SEA,133,OAK
1,318969,2012-03-29,Seattle Mariners,Oakland Athletics,72,6:04 PM,136,SEA,133,OAK
2,317733,2012-04-04,St. Louis Cardinals,Miami Marlins,79,7:15 PM,138,STL,146,MIA
3,317735,2012-04-05,Boston Red Sox,Detroit Tigers,43,1:10 PM,111,BOS,116,DET
4,317734,2012-04-05,Atlanta Braves,New York Mets,53,1:18 PM,144,ATL,121,NYM
...,...,...,...,...,...,...,...,...,...,...
4747,349168,2013-09-29,San Diego Padres,San Francisco Giants,67,1:06 PM,135,SD,137,SF
4748,349162,2013-09-29,Colorado Rockies,Los Angeles Dodgers,85,1:10 PM,115,COL,119,LA
4749,349169,2013-09-29,Washington Nationals,Arizona Diamondbacks,74,1:10 PM,120,WAS,109,ARI
4750,349173,2013-09-29,Oakland Athletics,Seattle Mariners,56,1:10 PM,133,OAK,136,SEA


In [ ]:
# Websrape yearly team batting data from baseball-reference.com
s = defaultdict(list)
try:
    for team in tqdm(df['teamabb_away'].unique()):
        if team == 'WAS':
            teamm = 'WSN'
        elif team == 'CWS':
            teamm = 'CHW'
        elif team == 'ANA':
            teamm = 'LAA'
        else:
            teamm = team
        for year in range (2014,2020,1):
            r = BeautifulSoup(re.get('https://www.baseball-reference.com/teams/'+teamm+'/'+str(year)+'-batting.shtml').content,'html.parser')
            for td in r.find_all('tr')[-3].find_all('td'):
                s[td['data-stat']].append(td.text)
            s['year'].append(year)
            s['team'].append(team)
            time.sleep(2)
except IndexError:
    print(team,year)

In [133]:
# Save that data
pd.DataFrame(s).to_csv('Data/teambatting.csv',index=False)

In [138]:
# Convert dictionary to dataframe
s = pd.DataFrame(s)

In [140]:
# Extract year from date
df['year'] = df['game_date'].str.extract(r'(\d{4})-\d{2}-\d{2}').astype(int)

In [144]:
# Merge to get yearly team batting info
df = df.merge(s,left_on=['year','teamabb_away'],right_on=['year','team']).merge(s,left_on=['year','teamabb_home'],right_on=['year','team'],suffixes=['_away','_home'])

In [12]:
# Get both starting away and home pitchers data
pa = defaultdict(list)
ph = defaultdict(list)
for gameid in tqdm(df['game_id'].unique()):
    for key, value in statsapi.player_stat_data(statsapi.boxscore_data(gameid)['awayPitchers'][1]['personId'], group='pitching',type='career')['stats'][0]['stats'].items():
        pa[key].append(value)
        
    for key, value in statsapi.player_stat_data(statsapi.boxscore_data(gameid)['homePitchers'][1]['personId'], group='pitching',type='career')['stats'][0]['stats'].items():
        ph[key].append(value)
        
    pa['game_id'].append(gameid)
    ph['game_id'].append(gameid)

100%|████████████████████████████████████████████████████████████████████████████| 4752/4752 [1:46:35<00:00,  1.35s/it]


In [153]:
# Save it
pd.DataFrame(pa).to_csv('Data/awaypitchers',index=False)

In [154]:
# Save it
pd.DataFrame(ph).to_csv('Data/homepitchers',index=False)

In [19]:
# Gather the runs data for all games
runs = {'gameid':[], 'awayruns':[], 'homeruns':[]}
for gameid in tqdm(df['game_id'].unique()):
    game = statsapi.boxscore_data(gameid)
    runs['awayruns'].append(game['away']['teamStats']['batting'])
    runs['homeruns'].append(game['home']['teamStats']['batting'])
    runs['gameid'].append(gameid)

100%|██████████████████████████████████████████████████████████████████████████████| 4752/4752 [53:57<00:00,  1.47it/s]


In [20]:
# convert dictionary to dataframe
runs = pd.DataFrame(runs)

In [22]:
# Reformat it 
runs['awayruns'] = runs['awayruns'].apply(lambda x: x['runs'])
runs['homeruns'] = runs['homeruns'].apply(lambda x: x['runs'])

In [23]:
# compute the total runs
runs['total'] = runs['awayruns'] + runs['homeruns']

In [24]:
# compute the run difference
runs['diff'] = runs['awayruns'] - runs['homeruns']

In [25]:
runs

,gameid,awayruns,homeruns,total,diff
0,317775,3,1,4,2
1,318969,1,4,5,-3
2,317733,4,1,5,3
3,317735,2,3,5,-1
4,317734,0,1,1,-1
...,...,...,...,...,...
4747,349168,6,7,13,-1
4748,349162,2,1,3,1
4749,349169,2,3,5,-1
4750,349173,9,0,9,9


In [31]:
# Save it
runs.to_csv('Data/runs.csv',index=False)